https://abvijaykumar.medium.com/fine-tuning-llm-parameter-efficient-fine-tuning-peft-lora-qlora-part-2-d8e23877ac6f

In [ ]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops
!pip install -q wandb

In [ ]:
from datasets import load_dataset
from random import randrange

import torch
import wandb
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, AutoPeftModelForCausalLM
from trl import SFTTrainer
from huggingface_hub import login


Let us understand why we need these various dependencies

- trl: This Python package “Transformer Reinforcement Learning” is used for fine-tuning the transformer model, using reinforcement learning. We will use our instruction dataset to perform this reinforcement learning and fine-tune the model. We will be using SFTrainer object to perform the fine-tuning.

- transformers: This package provides all the APIs for downloading and working with various pre-trained models that are in the huggingface model hub. In our example, we will be downloading Salesforce/codegen-350M-mono. We will also be using the bits and bytes library from transformers, for quantization and AutoTokenizers for creating a tokenizer for the pre-trained model.

- accelerate: This is another very powerful huggingface package, that hides the complexity of the developer trying to write/manage code needed to use multi-GPUs/TPU/fp16.

- peft: This package provides all the APIs we will need to perform the LoRA technique.

- datasets: This huggingface package provides access to the various datasets in the huggingface hub.

- wandb: This library provides access to the Weights and Biases library to capture various metrics, during the fine-tuning process.

In [ ]:
model_name = "Salesforce/codegen-350M-mono"
dataset_name = "iamtarun/python_code_instructions_18k_alpaca"
device_map = {"": 0}

In [ ]:
peft_config = LoraConfig(
      lora_alpha=16,
      lora_dropout=0.1,
      r=64,
      bias="none",
      task_type="CAUSAL_LM",
)

The LoraConfig has the following attributes.

- lora_alpha: scaling factor for the weight matrices. alpha is a scaling factor that adjusts the magnitude of the combined result (base model output + low-rank adaptation). We have set it to 16. You can find more details of this in the LoRA paper here.

- lora_dropout: dropout probability of the LoRA layers. This parameter is used to avoid overfitting. This technique basically drop-outs some of the neurons during both forward and backward propagation, this will help in removing dependency on a single unit of neurons. We are setting this to 0.1 (which is 10%), which means each neuron has a dropout chance of 10%.

- r: This is the dimension of the low-rank matrix, Refer to Part 1 of this blog for more details. In this case, we are setting this to 64 (which effectively means we will have 512x64 and 64x512 parameters in our LoRA adapter.

- bias: We will not be training the bias in this example, so we are setting that to “none”. If we have to train the biases, we can set this to “all”, or if we want to train only the LORA biases then we can use “lora_only”

- task_type: Since we are using the Causal language model, the task type we set to CAUSAL_LM.

Source: https://medium.com/@fartypantsham/what-rank-r-and-alpha-to-use-in-lora-in-llm-1b4f025fd133


alpha = rank is scaling weights at 1.0

What you train in LORA weights will be then merged with the main weights of model at x 1.0

Previously people were suggesting alpha = (2 x rank), which is like yelling at your model really loud — all in order to make the newly learned weights “louder” than the model’s own. That requires a really good and large dataset, otherwise you are just amplifying nonsense.

The model knows how to speak well already, while your dataset is too small to teach (or scream at) the model any language fundamentals. Increasing alpha amplifies everything, not just the stuff you wish the model learns from it.

I would suggest rank = alpha, most of the time as your base — because it is very easily to attenuate the LORA data after the training is done if it appears to be too “loud”, overtaking the entire model.

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16"
)

- load_in_4bit: we are loading the base model with a 4-bit quantization, so we are setting this value to True.

- bnb_4bit_use_double_quant: We also want double quantization so that even the quantization constant is quantized. So we are setting this to True.

- bnb_4bit_quant_type: We are setting this to nf4.

- bnb_4bit_compute_dtype: and the compute datatype we are setting to float16


In [1]:
def prompt_instruction_format(sample):
    return f"""### Instruction:
    Use the Task below and the Input given to write the Response, which is a programming code that can solve the following Task:

    ### Task:
    {sample['instruction']}

    ### Input:
    {sample['input']}

    ### Response:
    {sample['output']}
    """

In [ ]:
dataset = load_dataset(dataset_name, split=split)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name,
          quantization_config=bnb_config,
          use_cache = False,
          device_map=device_map)
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
trainingArgs = TrainingArguments(
    output_dir=finetunes_model_name,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    logging_steps=5,
    save_strategy="epoch",
    learning_rate=2e-4,
    weight_decay=0.001,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    group_by_length=False,
    lr_scheduler_type="cosine",
    disable_tqdm=True,
    report_to="wandb",
    seed=42
)

- output_dir: Output directory where the model predictions and checkpoints will be stored
- num_train_epochs=3: Number of training epochs
- per_device_train_batch_size=4: Batch size per GPU for training
- gradient_accumulation_steps=2: Number of update steps to accumulate the gradients for
- gradient_checkpointing=True: Enable gradient checkpointing. Gradient checkpointing is a technique used to reduce memory consumption during the training of deep neural networks, especially in situations where memory usage is a limiting factor. Gradient checkpointing selectively re-computes intermediate activations during the backward pass instead of storing them all, thus performing some extra computation to reduce memory usage.
- optim=”paged_adamw_32bit”: Optimizer to use, We will be using paged_adamw_32bit
- logging_steps=5: Log on to the console on the progress every 5 steps.
- save_strategy=”epoch”: save after every epoch
- learning_rate=2e-4: Learning rate
- weight_decay=0.001: Weight decay is a regularization technique used while training the models, to prevent overfitting by adding a penalty term to the loss function. Weight decay works by adding a term to the loss function that penalizes large values of the model’s weights.
- max_grad_norm=0.3: This parameter sets the maximum gradient norm for gradient clipping.
- warmup_ratio=0.03: The warm-up ratio is a value that determines what fraction of the total training steps or epochs will be used for the warm-up phase. In this case, we are setting it to 3%. Warm-up refers to a specific learning rate scheduling strategy that gradually increases the learning rate from its initial value to its full value over a certain number of training steps or epochs.
- lr_scheduler_type=”cosine”: Learning rate schedulers are used to adjust the - learning rate dynamically during training to help improve convergence and model performance. We will be using the cosine type for the learning rate scheduler.
- report_to=”wandb”: We want to report our metrics to Weights and Bias
- seed=42: This is the random seed that is set during the beginning of the training.

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length=2048,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=prompt_instruction_format,
    args=trainingArgs,
)

In [ ]:
trainer.train()

In [ ]:
# Merge LoRA with the base model and save the merged model
merged = trained_model.merge_and_unload()
merged.save_pretrained("merged",safe_serialization=True)
tokenizer.save_pretrained("merged")

#push merged model to the hub
merged.push_to_hub("codegen-350M-mono-python-18k-alpaca")
tokenizer.push_to_hub("codegen-350M-mono-python-18k-alpaca")